# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPy/weather_data.csv")
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,mar del plata,-38.00,-57.56,75.00,69,75,16.11,AR,1607455285
1,bredasdorp,-34.53,20.04,64.40,88,4,14.99,ZA,1607455416
2,mataura,-46.19,168.86,57.99,69,86,5.01,NZ,1607455417
3,kavieng,-2.57,150.80,82.40,72,100,4.99,PG,1607455418
4,jieshi,22.81,115.83,64.00,69,70,1.01,CN,1607455419


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = cities_df[['Lat','Lng']]
humidity = cities_df['Humidity']
print(len(humidity))

554


In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
vacation_df = cities_df.loc[(cities_df['Max Temp']>72) & (cities_df['Wind Speed']<10) \
                            & (cities_df['Max Temp']<78) & (cities_df['Humidity']<70) \
                            & (cities_df['Cloudiness']<5)]

In [19]:
### Hotel Map
# * Store into variable named `hotel_df`.
# * Add a "Hotel Name" column to the DataFrame.
# * Set parameters to search for hotels with 5000 meters.
# * Hit the Google Places API for each city's coordinates.
# * Store the first Hotel result into the DataFrame.
# * Plot markers on top of the heatmap.

In [20]:
vacation_df['Hotel'] = ""

#vacation_df.drop(vacation_df[vacation_df['City'] == "agadez"].index, inplace=True)
vacation_df

<ipython-input-20-7c40fe4ad9b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacation_df['Hotel'] = ""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
32,bilma,18.69,12.92,75.61,18,0,5.91,NE,1607455451,
48,castro,-24.79,-50.01,77.81,57,1,7.74,BR,1607455396,
49,lompoc,34.64,-120.46,77.00,29,1,5.82,US,1607455375,
90,umm lajj,25.02,37.27,76.50,41,0,7.14,SA,1607455520,
241,aswan,24.09,32.91,74.82,22,0,6.78,EG,1607455695,
313,opuwo,-18.06,13.84,77.50,25,0,9.04,NaN,1607455778,
385,agadez,19.75,10.25,73.36,21,0,6.26,NE,1607455873,
398,mecca,21.43,39.83,75.81,36,0,0.38,SA,1607455888,
520,manutuke,-38.68,177.92,77.00,54,1,8.01,NZ,1607456030,


In [21]:
#check iterrow logic
for index, row in vacation_df.iterrows():
    location =  "\"%s\"" % f"{row['Lat']}, {row['Lng']}"
    print(location)
    print(f"{index}: {row['City']}")
    #vacation_df.loc[index, 'location'] = str(location)
    #vacation_df['location'] = vacation_df['location'].astype(str)

"18.69, 12.92"
32: bilma
"-24.79, -50.01"
48: castro
"34.64, -120.46"
49: lompoc
"25.02, 37.27"
90: umm lajj
"24.09, 32.91"
241: aswan
"-18.06, 13.84"
313: opuwo
"19.75, 10.25"
385: agadez
"21.43, 39.83"
398: mecca
"-38.68, 177.92"
520: manutuke


In [24]:
#Find hotels
for index, row in vacation_df.iterrows():
    # set up a parameters dictionary
    location = f"{row['Lat']}, {row['Lng']}"
    params = {
        "location": location,
        "radius": 5000,
        "type": "lodging",
        "key": g_key
    }
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # run a request using our params dictionary
    try:
        response = requests.get(base_url, params=params)
        places_data = response.json()
        #print(json.dumps(places_data, indent=4, sort_keys=True))
        print(places_data["results"][0]["name"])
        vacation_df.loc[index, 'Hotel'] = places_data["results"][0]["name"]
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        vacation_df.loc[index, 'Hotel'] = "No Hotel found"

Expeditions Ténére voyages


C:\Users\melanie.nolker\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


CHACARA BAILLY
Holiday Inn Express Lompoc
Moon light Furnished Units
Nuba Nile
ABBA Guesthouse
Missing field/result... skipping.
Hotel Fairmont Makkah Clock Royal Tower
Jolly Stockman Hotel


In [25]:
vacation_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
32,bilma,18.69,12.92,75.61,18,0,5.91,NE,1607455451,Expeditions Ténére voyages
48,castro,-24.79,-50.01,77.81,57,1,7.74,BR,1607455396,CHACARA BAILLY
49,lompoc,34.64,-120.46,77.00,29,1,5.82,US,1607455375,Holiday Inn Express Lompoc
90,umm lajj,25.02,37.27,76.50,41,0,7.14,SA,1607455520,Moon light Furnished Units
241,aswan,24.09,32.91,74.82,22,0,6.78,EG,1607455695,Nuba Nile
313,opuwo,-18.06,13.84,77.50,25,0,9.04,NaN,1607455778,ABBA Guesthouse
385,agadez,19.75,10.25,73.36,21,0,6.26,NE,1607455873,No Hotel found
398,mecca,21.43,39.83,75.81,36,0,0.38,SA,1607455888,Hotel Fairmont Makkah Clock Royal Tower
520,manutuke,-38.68,177.92,77.00,54,1,8.01,NZ,1607456030,Jolly Stockman Hotel


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
locations = vacation_df[["Lat", "Lng"]] 

In [34]:
hotel_test = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
hotel_test

['\n<dl>\n<dt>Name</dt><dd>Expeditions Ténére voyages</dd>\n<dt>City</dt><dd>bilma</dd>\n<dt>Country</dt><dd>NE</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>CHACARA BAILLY</dd>\n<dt>City</dt><dd>castro</dd>\n<dt>Country</dt><dd>BR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Holiday Inn Express Lompoc</dd>\n<dt>City</dt><dd>lompoc</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Moon light Furnished Units</dd>\n<dt>City</dt><dd>umm lajj</dd>\n<dt>Country</dt><dd>SA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Nuba Nile</dd>\n<dt>City</dt><dd>aswan</dd>\n<dt>Country</dt><dd>EG</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>ABBA Guesthouse</dd>\n<dt>City</dt><dd>opuwo</dd>\n<dt>Country</dt><dd>nan</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>No Hotel found</dd>\n<dt>City</dt><dd>agadez</dd>\n<dt>Country</dt><dd>NE</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel Fairmont Makkah Clock Royal Tower</dd>\n<dt>City</dt><dd>mecca</dd>\n<dt>Country</dt><dd>SA</dd>\n</dl>\n',
 '\n<dl>

In [35]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))